In [1]:
#download transfer market data
#try and download whoscored
from  urllib.request import Request,urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from IPython.display import display

In [2]:

years =['2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016']
def doItAll():
    did_it= map(getFile,years)
    for x in did_it:
        print('we did it')

def getFile(year):
    url = makeUrl(year)
    filename = makeFilename(year)
    request = Request(url)
    request.add_header('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/56.0.2924.87 Safari/537.36')
    try:
        response = urlopen(request)
        webContent = response.read()
        f = open(filename, 'wb')
        f.write(webContent)
        f.close
    except urllib.error.URLError as e:
        print (e)
        print (e.headers)
    return 
    
    

    
def makeUrl(year):
    url = 'http://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/saison_id/'+year
    return url
    
def makeFilename(year):
    filename = "data\html\{}".format(year)
    return filename


    
    

In [113]:
doItAll()

we did it
we did it
we did it
we did it
we did it
we did it
we did it
we did it
we did it
we did it
we did it
we did it


In [7]:
def soupNazi(file):
    data_rows=[]
    nail = open(file,'r',encoding="latin-1")
    hammer = nail.read()
    minestrone = BeautifulSoup(hammer,"html.parser")
    boxes = minestrone.find_all("div",{"class":"box"})
    for box in boxes:
        headers = box.find_all("div",{"class":"table-header"})
        team = None
        for header in headers:        
            header_anchors = header.find_all("a",{"class":"vereinprofil_tooltip"})
            for anch in header_anchors:
                if anch.string:
                    team = anch.string
        if team:
            bodies = box.find_all("tbody")
            for idx,bod in enumerate(bodies):
                rows = bod.find_all("tr")
                #first body is transfers in second row is transfers out
                for row in rows:
                    if idx ==0:
                        movement ="in"
                    else:
                        movement ="out"
                    tds = row.find_all("td")
                    if len(tds)==0:
                        continue;
                    try:
                        name = tds[0].find_all("a")[0].string
                        age = tds[1].string
                        nationality =tds[2].find_all("img")[0]["title"]
                        pos = tds[4].string
                        market = tds[5].string
                        trader = tds[7].find("a").string
                        fee = tds[8].string or (tds[8].find("i").string if tds[8].find("i") else "-")
                        data_row=[team,name,age,nationality,pos,market,movement,trader,fee]
                        data_rows.append(data_row)
                    except Exception:
                        print( tds[8])
                        raise
    frame = pd.DataFrame(data_rows,columns=["team","name","age","nationality","pos","market","direction","trader","fee"])
    year = file.split("\\")[2]
    print (year)
    frame.to_csv("data\\csv\\"+"transfer_"+year+".csv")
    
    

def beautiful():
    for x in years:
        fname = makeFilename(x)
        soupNazi(fname)
    
beautiful()

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [15]:
ohfive =pd.read_csv("data\\csv\\transfer_2005.csv",encoding="latin-1",index_col=0,dtype={"age":np.float32},na_values=["-"])

In [27]:
display(ohfive[ohfive["age"]>20].groupby(["team","direction"],as_index=False).aggregate(["mean","count"]))

age      
                                     mean count
team                 direction                 
Arsenal FC           in         23.400000    10
                     out        23.272728    11
Aston Villa          in         26.125000     8
                     out        24.545454    11
Birmingham City      in         24.916666    12
                     out        26.312500    16
Blackburn Rovers     in         23.125000     8
                     out        25.307692    13
Bolton Wanderers     in         28.545454    11
                     out        30.916666    12
Charlton Athletic    in         26.375000    16
                     out        27.000000    14
Chelsea FC           in         24.909090    11
                     out        24.000000    16
Everton FC           in         26.272728    11
                     out        27.000000     8
Fulham FC            in         27.000000    18
                     out        27.733334    15
Liverpool FC         in         23.875000    16
                     out        24.772728    22
Manchester City      in         26.375000     8
                     out        26.600000    10
Manchester United    in         23.666666    12
                     out        24.928572    14
Middlesbrough FC     in         24.375000     8
                     out        29.000000     6
Newcastle United     in         27.500000    10
                     out        25.666666    12
Portsmouth FC        in         26.148148    27
                     out        27.038462    26
Sunderland AFC       in         24.866667    15
                     out        25.842106    19
Tottenham Hotspur    in         24.133333    15
                     out        24.000000    17
West Bromwich Albion in         24.916666    12
                     out        26.588236    17
West Ham United      in         23.809525    21
                     out        27.263159    19
Wigan Athletic       in         28.500000    14
                     out        28.071428    14